## IBM/Coursera Capstone Project

### This is the notebook I've used to find the best neighborhood in my hometown (Curitiba, Brazil) for my client to open a vegan restaurant

### Importing all the necessary libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


### Transforming a .csv file with Curitiba's neighborhoods and coordinates into a pandas DF

In [2]:
df = pd.read_csv('C:/Users/Andre/Documents/IBM/09_Capstone/Project/CuritibaNeighborhoods.csv')
df.head()

,Neighborhood,Latitude,Longitude
0,Abranches,-25.36147,-49.27205
1,Ahu,-25.39984,-49.26194
2,Alto Boqueirao,-25.53267,-49.23907
3,Alto da Gloria,-25.41945,-49.26215
4,Alto da XV,-25.42807,-49.25119


### Getting info on how many neighborhoods exist in the DB

In [3]:
# Getting the number of boroughs and neighborhoods in the DB
df.shape

(75, 3)

#### Using geopy library to get the latitude and longitude values of Curitiba

In [4]:
# Defining the user_agent as curitiba_explorer and getting the coordinates for Curitiba
address = 'Curitiba, BR'

geolocator = Nominatim(user_agent="curitiba_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Curitiba are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Curitiba are -25.4295963, -49.2712724.


#### Creating a map of Curitiba with neighborhoods superimposed on top

In [5]:
# create map of New York using latitude and longitude values
map_curitiba = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_curitiba)  
    
map_curitiba

#### Define Foursquare Credentials and Version

In [6]:
CLIENT_ID = 'HCBL5FEAAFXXWORLJQL3INMDHKNVFRQCELGX0FSKUPRV4U4V' # your Foursquare ID
CLIENT_SECRET = '1RYZI1VIFOXO5JVP4BEJX55KY1RE3EJA1LEIH0UIW4RYZTAK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

#### Creating a function to find the top 100 venues within a radius of 500 meters

In [7]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Running the above function on each neighborhood and creating a new dataframe called curitiba_venues

In [8]:
curitiba_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Abranches
Ahu
Alto Boqueirao
Alto da Gloria
Alto da XV
Atuba
Augusta
Agua Verde
Bacacheri
Bairro Alto
Barreirinha
Batel
Bigorrilho
Boa Vista
Bom Retiro
Boqueirao
Butiatuvinha
Cabral
Cachoeira
Cajuru
Campina do Siqueira
Campo Comprido
Campo de Santana
Capao da Imbuia
Capao Raso
Cascatinha
Caximba
Centro
Centro Civico
Cidade Industrial
Cristo Rei
Fanny
Fazendinha
Ganchinho
Guaira
Guabirotuba
Hauer
Hugo Lange
Jardim Botanico
Jardim Social
Jd. das Americas
Juveve
Lamenha Pequena
Lindoia
Merces
Mossungue
Novo Mundo
Orleans
Parolin
Pilarzinho
Pinheirinho
Portao
Prado Velho
Reboucas
Riviera
Sao Braz
Sao Francisco
Sao Joao
Sao Miguel
Santa Candida
Santa Felicidade
Santa Quiteria
Santo Inacio
Sao Lourenco
Seminario
Sitio Cercado
Taboao
Taruma
Tatuquara
Tingui
Uberaba
Umbara
Vila Izabel
Vista Alegre
Xaxim


In [9]:
# Printing the head of the list of venues by Neighborhood, and measuring the size of the DB
print(curitiba_venues.shape)
curitiba_venues.head()

(1429, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Abranches,-25.36147,-49.27205,Anjo Da Bola Sports,-25.363988,-49.268761,Soccer Field
1,Abranches,-25.36147,-49.27205,Colletti Atacado Calçados,-25.360542,-49.274255,Department Store
2,Abranches,-25.36147,-49.27205,Louie Records,-25.359984,-49.270021,Concert Hall
3,Abranches,-25.36147,-49.27205,Chacara dos Carneirinhos,-25.364699,-49.274804,Farm
4,Ahu,-25.39984,-49.26194,Über Ale,-25.399820,-49.258883,Bar


#### Checking how many venues were returned by Neighborhood

In [10]:
curitiba_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Abranches,4,4,4,4,4,4
Agua Verde,32,32,32,32,32,32
Ahu,21,21,21,21,21,21
Alto Boqueirao,5,5,5,5,5,5
Alto da Gloria,36,36,36,36,36,36
Alto da XV,41,41,41,41,41,41
Atuba,4,4,4,4,4,4
Bacacheri,25,25,25,25,25,25
Bairro Alto,7,7,7,7,7,7


### Analyzing each Neighborhood

In [11]:
# one hot encoding
curitiba_onehot = pd.get_dummies(curitiba_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
curitiba_onehot['Neighborhood'] = curitiba_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [curitiba_onehot.columns[-1]] + list(curitiba_onehot.columns[:-1])
curitiba_onehot = curitiba_onehot[fixed_columns]

curitiba_onehot.head()

,Yoga Studio,Acai House,Accessories Store,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Court,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Campground,Candy Store,Carpet Store,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Service,Event Space,Exhibit,Factory,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Government Building,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Housing Development,Ice Cream Shop,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lawyer,Leather Goods Store,Library,Liquor Store,Lottery Retailer,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mineiro Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Moving Target,Museum,Music School,Music Venue,Neighborhood,Nightclub,Office,Paintball Field,Paper / Office Supplies Store,Park,Pastelaria,Pastry Shop,Perfume Shop,Pet Store,Pharmacy,Photography Lab,Piano Bar,Pie Shop,Pizza Place,Plaza,Pool,Pool Hall,Portuguese Restaurant,Pub,Public Art,Racetrack,Recording Studio,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Slovak Restaurant,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,South American Restaurant,Southern Brazilian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Street Art,Street Fair,Supermarket,Surf Spot,Sushi Restaurant,Tailor Shop,Tattoo Parlor,Tea Room,Tennis Court,Theater,Theme Park,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Used Bookstore,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Warehouse Store,Water Park,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Abranches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Abranches,0

In [12]:
# Shape of database after one hot encoding
curitiba_onehot.shape

(1429, 232)

In [13]:
# Grouping rows by neighborhood and taking the mean of the frequency of occurrence of each category,
# to compact the size of the DB
curitiba_grouped = curitiba_onehot.groupby('Neighborhood').mean().reset_index()
curitiba_grouped

,Neighborhood,Yoga Studio,Acai House,Accessories Store,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Court,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Campground,Candy Store,Carpet Store,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Service,Event Space,Exhibit,Factory,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Government Building,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Housing Development,Ice Cream Shop,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lawyer,Leather Goods Store,Library,Liquor Store,Lottery Retailer,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mineiro Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Moving Target,Museum,Music School,Music Venue,Nightclub,Office,Paintball Field,Paper / Office Supplies Store,Park,Pastelaria,Pastry Shop,Perfume Shop,Pet Store,Pharmacy,Photography Lab,Piano Bar,Pie Shop,Pizza Place,Plaza,Pool,Pool Hall,Portuguese Restaurant,Pub,Public Art,Racetrack,Recording Studio,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Slovak Restaurant,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,South American Restaurant,Southern Brazilian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Street Art,Street Fair,Supermarket,Surf Spot,Sushi Restaurant,Tailor Shop,Tattoo Parlor,Tea Room,Tennis Court,Theater,Theme Park,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Used Bookstore,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Warehouse Store,Water Park,Wine Bar,Wine Shop,Women's Store
0,Abranches,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.25,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0

In [14]:
# Checking the size of the new DB
curitiba_grouped.shape

(68, 232)

#### Printing each neighborhood along with the top 5 most common venues

In [15]:
num_top_venues = 5

for hood in curitiba_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = curitiba_grouped[curitiba_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Abranches----
              venue  freq
0      Concert Hall  0.25
1  Department Store  0.25
2      Soccer Field  0.25
3              Farm  0.25
4       Yoga Studio  0.00


----Agua Verde----
                  venue  freq
0  Brazilian Restaurant  0.09
1           Pizza Place  0.09
2            Beer Store  0.06
3                Bakery  0.03
4          Soccer Field  0.03


----Ahu----
          venue  freq
0    Restaurant  0.14
1   Pizza Place  0.10
2    Steakhouse  0.10
3     BBQ Joint  0.05
4  Dance Studio  0.05


----Alto Boqueirao----
                  venue  freq
0  Gym / Fitness Center   0.4
1             Irish Pub   0.2
2               Stadium   0.2
3           Supermarket   0.2
4           Yoga Studio   0.0


----Alto da Gloria----
                  venue  freq
0                  Café  0.08
1                Buffet  0.06
2           Coffee Shop  0.06
3        Chocolate Shop  0.06
4  Brazilian Restaurant  0.06


----Alto da XV----
                  venue  freq
0            Resta

                  venue  freq
0                Bakery  0.12
1          Soccer Field  0.09
2  Brazilian Restaurant  0.09
3          Churrascaria  0.06
4           Pizza Place  0.06


----Santo Inacio----
                   venue  freq
0         Rental Service  0.33
1  General Entertainment  0.33
2         Breakfast Spot  0.33
3           Perfume Shop  0.00
4          Moving Target  0.00


----Sao Braz----
                    venue  freq
0    Gym / Fitness Center  0.12
1             Pizza Place  0.12
2  Furniture / Home Store  0.06
3                Pharmacy  0.06
4            Burger Joint  0.06


----Sao Francisco----
       venue  freq
0        Bar  0.21
1  Nightclub  0.06
2      Plaza  0.05
3  Rock Club  0.05
4       Café  0.04


----Sao Lourenco----
                  venue  freq
0            Acai House  0.14
1               Theater  0.14
2          Dessert Shop  0.07
3  Arts & Entertainment  0.07
4                  Park  0.07


----Seminario----
                  venue  freq
0        

#### Putting all this info in a new Pandas DF

In [16]:
# Writing a function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [17]:
# Creating the new dataframe and displaying the top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = curitiba_grouped['Neighborhood']

for ind in np.arange(curitiba_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(curitiba_grouped.iloc[ind, :], num_top_venues)

cwb = neighborhoods_venues_sorted.set_index('Neighborhood')
cwb

,1,2,3,4,5,6,7,8,9,10
Neighborhood,,,,,,,,,,
Abranches,Concert Hall,Department Store,Soccer Field,Farm,Exhibit,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market
Agua Verde,Pizza Place,Brazilian Restaurant,Beer Store,Dance Studio,Bakery,Martial Arts Dojo,Soccer Field,Burger Joint,Sandwich Place,Breakfast Spot
Ahu,Restaurant,Pizza Place,Steakhouse,Burger Joint,Italian Restaurant,BBQ Joint,Café,Bar,Sandwich Place,Dessert Shop
Alto Boqueirao,Gym / Fitness Center,Stadium,Irish Pub,Supermarket,Women's Store,Exhibit,Food & Drink Shop,Food,Flower Shop,Flea Market
Alto da Gloria,Café,Brazilian Restaurant,Buffet,Soccer Stadium,Chocolate Shop,Coffee Shop,Yoga Studio,Portuguese Restaurant,Chinese Restaurant,Clothing Store
Alto da XV,Restaurant,Brazilian Restaurant,Bar,Bakery,Dessert Shop,Gym / Fitness Center,Gym,Chinese Restaurant,Sporting Goods Shop,Juice Bar
Atuba,Warehouse Store,Restaurant,Bus Station,Café,Women's Store,Event Space,Food,Flower Shop,Flea Market,Fish Market
Bacacheri,Bar,Pizza Place,Hot Dog Joint,Pharmacy,Paper / Office Supplies Store,Brewery,Fruit & Vegetable Store,Liquor Store,Sandwich Place,Beer Store
Bairro Alto,Pizza Place,BBQ Joint,Pet Store,Restaurant,Bakery,Martial Arts Dojo,Farmers Market,Exhibit,Factory,Farm


### Finding the ideal neighborhood where to open a vegan restaurant

Here I have decided to filter the candidate neighborhoods. First thing, I've filtered the column "1st most common venue"  with the word "Restaurant", so I could find neighborhoods traditionally populated with restaurants.

In [18]:
cwb1 = cwb[cwb['1'].str.contains("Restaurant")]
cwb1

,1,2,3,4,5,6,7,8,9,10
Neighborhood,,,,,,,,,,
Ahu,Restaurant,Pizza Place,Steakhouse,Burger Joint,Italian Restaurant,BBQ Joint,Café,Bar,Sandwich Place,Dessert Shop
Alto da XV,Restaurant,Brazilian Restaurant,Bar,Bakery,Dessert Shop,Gym / Fitness Center,Gym,Chinese Restaurant,Sporting Goods Shop,Juice Bar
Campina do Siqueira,Brazilian Restaurant,Gym / Fitness Center,Yoga Studio,Restaurant,Burger Joint,Motorcycle Shop,Café,Beer Bar,Supermarket,Sandwich Place
Campo Comprido,Middle Eastern Restaurant,Pizza Place,Gym,Bakery,Women's Store,Exhibit,Food & Drink Shop,Food,Flower Shop,Flea Market
Cristo Rei,Restaurant,Pizza Place,Brazilian Restaurant,Pharmacy,Café,Massage Studio,Gym,Rock Club,Steakhouse,Clothing Store
Hauer,Brazilian Restaurant,Restaurant,Tea Room,Hot Dog Joint,Fast Food Restaurant,Chinese Restaurant,Pet Store,Snack Place,Food,Buffet
Jardim Botanico,Restaurant,Park,Racetrack,Chinese Restaurant,Bakery,Farmers Market,Tea Room,Tennis Court,Athletics & Sports,Music Venue
Merces,Middle Eastern Restaurant,Pharmacy,Pizza Place,Café,Dessert Shop,Brazilian Restaurant,Scenic Lookout,Restaurant,Supermarket,Buffet
Pinheirinho,Restaurant,Bakery,BBQ Joint,Grocery Store,Hardware Store,Gym,Fast Food Restaurant,Women's Store,Farmers Market,Factory


The next step was to find, among these filtered neighborhoods, the ones that include other venues related to the vegan public. This is a way to find a hot spot where the vegan public may gather around. In other words, the perfect spot for my client's restaurant. So I decided that another strong venue that would spot the presence of the vegan community would be a Farmers Market.

Couldn't find it in the "2nd most frequent venue" column, as seen below. So I've moved on to the 3rd column, as so on, until I find a Farmers Market in a neighborhood populated with restaurants.

In [19]:
cwb2 = cwb1[cwb1['2'].str.contains("Farmers Market")]
cwb2

,1,2,3,4,5,6,7,8,9,10
Neighborhood,,,,,,,,,,


In [20]:
cwb3 = cwb1[cwb1['3'].str.contains("Farmers Market")]
cwb3

,1,2,3,4,5,6,7,8,9,10
Neighborhood,,,,,,,,,,


There you go! In the 4th column, I've found the presence of a Farmers Market in a neighborhood well populated with restaurants. That will be the target neighborhood to my client's restaurant: Santa Felicidade (portuguese for "Holy Happiness").
Also, by analysing the other columns, we can see that this is a neighborhood that consists entirely of food venues. Looks just like the perfect spot.

In [21]:
cwb4 = cwb1[cwb1['4'].str.contains("Farmers Market")]
cwb4

,1,2,3,4,5,6,7,8,9,10
Neighborhood,,,,,,,,,,
Santa Felicidade,Italian Restaurant,Bakery,Café,Farmers Market,Snack Place,Ice Cream Shop,Chocolate Shop,Bar,Gift Shop,Restaurant
